In [1]:
import pandas as pd
import numpy as np

In [2]:
queries = ["What is the Sorcerer’s Stone?",
           "How does Harry get into Gryffindor?",
           "What does the Mirror of Erised show Harry?",
           "Who helps Harry get past Fluffy?",
           "What happens during Harry’s first Quidditch match?", ]
complex_queries = [
    "How do Harry, Ron, and Hermione manage to bypass each of the obstacles guarding the Sorcerer’s Stone?",
    "what role does Fluffy play in the protection system?",
    "Snape’s and Quirrell’s interference Harrys first quidditch match"]

In [14]:
tf_idf_results_file = 'data/idx_passages_retrieved_TFIDF_k10.csv'
semantic_search_file = 'data/idx_passages_retrieved_IVF_nprob2.csv'
passages_df_file = 'data/clean_chunks.csv'

tf_idf_results = pd.read_csv(tf_idf_results_file).drop(columns=['Unnamed: 0'])
passages_df = pd.read_csv(passages_df_file)

In [48]:
for i, q_res in tf_idf_results.iterrows():
    print("===============================================================")
    print(f"Query: {queries[i]}")
    print("===============================================================")
    for j, rp in enumerate(passages_df.iloc[q_res].iterrows()):
        print(f"\tRetrieved Passage {j}")
        print(f"\t{rp[1]['str_idx']}")
        print(f"\t{rp[1]['chunk']}")
        print()

Query: What is the Sorcerer’s Stone?
	Retrieved Passage 0
	CHAPTER THIRTEEN
	There have been many reports of the Sorcerer's Stone over the centuries, but the only Stone currently in existence belongs to Mr. Nicolas Flamel, the noted alchemist and opera lover. Mr. Flamel, who celebrated his six hundred and sixty-fifth birthday last year, enjoys a quiet life in Devon with his wife, Perenelle (six hundred and fifty-eight).

"See?" said Hermione, when Harry and Ron had finished. "The dog must be guarding Flamel's Sorcerer's Stone! I bet he asked Dumbledore to keep it safe for him, because they're friends and he knew someone was after it, that's why he wanted the Stone moved out of Gringotts!"

"A stone that makes gold and stops you from ever dying!" said Harry. "No wonder Snape's after it! Anyone would want it."

"And no wonder we couldn't find Flamel in that Study of Recent Developments in Wizardry," said Ron. "He's not exactly recent if he's six hundred and sixty-five, is he?"

	Retrieve

# Issues with the TF-IDF Approach in Retrieval

TF-IDF is a widely used method for lexical-based retrieval, motivated by assigning weights to terms based on their frequency within a document and their rarity across the corpus. Seversl limitations of TF-IDF are:

1. TF-IDF relies purely on the occurrence of words and does not account for **synonyms, polysemy, or contextual meaning**. It treats words independently and does not capture relationships between different terms.

2. Since TF-IDF scores are based on exact term matches, it fails when queries contain different **inflections, stemming variations, or paraphrasing**.
3. TF-IDF defines a **high-dimensional vector space**, where each unique word in the corpus represents a separate dimension and often leads to **sparse representations**. The matrix becomes difficult to process efficiently as a fundtion of the vocabulary size. Also, short documents or queries term frequency is often low, which causes difficulties for small a corpus data.

Generally, any task including deeper-relations-understanding between the texts is less suited to be solved with /tf-/idf approaches.


In [12]:
def compare_approaches(k=5):
    tf_idf_results = pd.read_csv(tf_idf_results_file).drop(columns=['Unnamed: 0'])
    semantic_search = pd.read_csv(semantic_search_file).drop(columns=['Unnamed: 0'])

    for k in range(1, k + 1):
        recall_scores = []
        mrr_scores = []
        ndcg_scores = []
        chapters_mrr_scores = []
        chapters_recall_scores = []

        for i, query in enumerate(queries):
            actual = semantic_search.iloc[i].tolist()
            predicted = tf_idf_results.iloc[i].tolist()

            recall = recall_k(actual=actual, predicted=predicted, k=k)
            recall_scores.append(recall)

            mrr = mrr_k(actual=actual, predicted=predicted, k=k)
            mrr_scores.append(mrr)

            chapt_mrr = chapter_mrr(actual, predicted, k=k)
            chapters_mrr_scores.append(chapt_mrr)

            chapt_recall = chapter_recall(actual, predicted, k=k)
            chapters_recall_scores.append(chapt_recall)

            ndcg = ndcg_k(actual=actual, predicted=predicted, k=k)
            ndcg_scores.append(ndcg)

        avg_recall = np.mean(recall_scores)
        avg_mrr = np.mean(mrr_scores)
        avg_ndcg = np.mean(ndcg_scores)
        avg_crecall = np.mean(chapters_recall_scores)
        avg_cmrr = np.mean(chapters_mrr_scores)
        print(f"===================== k = {k} =====================")
        print(f"Max Recall@{k}: {np.max(recall_scores):.3f} (Query {np.argmax(recall_scores) + 1})")
        print(f"Max MRR@{k}: {np.max(mrr_scores):.3f} (Query {np.argmax(mrr_scores) + 1})")
        print(f"Max NDCG@{k}: {np.max(ndcg_scores):.3f} (Query {np.argmax(ndcg_scores) + 1})")
        print(f"Max C-Recall@{k}: {np.max(chapters_recall_scores):.3f} (Query {np.argmax(chapters_recall_scores) + 1})")
        print(f"Max CMRR@{k}: {np.max(chapters_mrr_scores):.3f} (Query {np.argmax(chapters_mrr_scores) + 1})\n")

        print(f"Avg Recall@{k}: {avg_recall:.3f}")
        print(f"Avg MRR@{k}: {avg_mrr:.3f}")
        print(f"Avg NDCG@{k}: {avg_ndcg:.3f}")
        print(f"Avg C-Recall@{k}: {avg_crecall:.3f}")
        print(f"Avg CMRR@{k}: {avg_cmrr:.3f}")
        print("================================================")


def recall_k(actual, predicted, k):
    """
     By increasing K to N or near N, we can return a perfect score every time, so relying solely on recall@K can be deceptive.
     Furthermore, it is an order-unaware metric.
    :param actual: Semantic search as "ground truth"
    :param predicted: Lexical retrieve with TF-IDF
    :param k: k relevance
    :return: recall@k scores
    """
    act_set = set(actual)
    pred_set = set(predicted[:k])
    result = round(len(act_set & pred_set) / float(len(act_set)), 4)
    return result


def mrr_k(actual, predicted, k):
    """
    Mean Reciprocal Rank (MRR) measures how early the first relevant result appears.
    :param actual: Semantic search as "ground truth"
    :param predicted: Lexical retrieve with TF-IDF
    :param k: Top-K results to consider
    :return: MRR@K score
    """
    cumm_r = 0
    for act in actual:
        for rank, item in enumerate(predicted[:k], start=1):
            if item == act:
                cumm_r += (1 / rank)
                return 1 / rank
    return cumm_r


def chapter_mrr(actual, predicted, k):
    """
    MRR@k score, adjusted for accounting Chapters instead of passages
    (By checking if the passages returned belongs to the same chapter)
    :param actual: Semantic search as "ground truth"
    :param predicted: Lexical retrieve with TF-IDF
    :param k: Top-K results to consider
    :return: Chapter_MRR@K score
    """
    actual_chapters = passages_df.iloc[actual]['str_idx'].tolist()
    pred_chapters = passages_df.iloc[predicted]['str_idx'][:k].tolist()
    cumm_mrr = 0
    for rank, chapter in enumerate(pred_chapters[:k], 1):
        if chapter in actual_chapters:
            # cumm_mrr += (1 / rank)
            return 1 / rank
    return cumm_mrr


def chapter_recall(actual, predicted, k):
    """
    Recall@k score, adjusted for accounting Chapters instead of passages.
    (By checking if the passages returned belongs to the same chapter)
    :param actual: Semantic search as "ground truth"
    :param predicted: Lexical retrieve with TF-IDF
    :param k: Top-K results to consider
    :return: Chapter_Recall@K score
    """
    actual_chapters = set(passages_df.iloc[actual]['str_idx'].tolist())
    pred_chapters = set(passages_df.iloc[predicted]['str_idx'][:k].tolist())
    result = round(len(actual_chapters & pred_chapters) / float(len(actual_chapters)), 4)
    return result


def ndcg_k(actual, predicted, k):
    """
    Normalized Discounted Cumulative Gain (NDCG) for ranking quality measure.
    :param actual: Semantic search as "ground truth"
    :param predicted: Lexical retrieve with TF-IDF
    :param k: Top-K results to consider
    :return: NDCG@K score
    """

    def dcg(scores):
        return sum((rel / np.log2(idx + 2)) for idx, rel in enumerate(scores))

    relevance_scores = [1 if item in actual else 0 for item in predicted[:k]]
    dcg_k = dcg(relevance_scores)
    idcg_k = dcg(sorted(relevance_scores, reverse=True))
    result = round(dcg_k / idcg_k, 4) if idcg_k > 0 else 0.0
    return result

In [13]:
passages_df = pd.read_csv(passages_df_file)
compare_approaches(k=5)

===================== k = 1 =====================
Max Recall@1: 0.200 (Query 1)
Max MRR@1: 1.000 (Query 1)
Max NDCG@1: 1.000 (Query 1)
Max C-Recall@1: 0.500 (Query 3)
Max CMRR@1: 1.000 (Query 1)

Avg Recall@1: 0.120
Avg MRR@1: 0.600
Avg NDCG@1: 0.600
Avg C-Recall@1: 0.233
Avg CMRR@1: 0.600
===================== k = 2 =====================
Max Recall@2: 0.400 (Query 1)
Max MRR@2: 1.000 (Query 1)
Max NDCG@2: 1.000 (Query 1)
Max C-Recall@2: 0.500 (Query 3)
Max CMRR@2: 1.000 (Query 1)

Avg Recall@2: 0.200
Avg MRR@2: 0.600
Avg NDCG@2: 0.600
Avg C-Recall@2: 0.323
Avg CMRR@2: 0.800
===================== k = 3 =====================
Max Recall@3: 0.400 (Query 1)
Max MRR@3: 1.000 (Query 1)
Max NDCG@3: 1.000 (Query 1)
Max C-Recall@3: 0.500 (Query 3)
Max CMRR@3: 1.000 (Query 1)

Avg Recall@3: 0.200
Avg MRR@3: 0.600
Avg NDCG@3: 0.600
Avg C-Recall@3: 0.323
Avg CMRR@3: 0.800
===================== k = 4 =====================
Max Recall@4: 0.400 (Query 1)
Max MRR@4: 1.000 (Query 1)
Max NDCG@4: 1.000 (Q

# Open Questions

#### Potential Improvements for Retrieval Using an LLM
Unlike traditional lexical-based approaches that rely only on exact term matching (TF-IDF), utilizing LLMs can capture contextual relationships between words, even when query and document vocabulary differ.

##### Potential Improvements:
1. Metadata indexing (Both Semantic Search and Lexical Retrieval): An interesting approach is to define more-comprehensive ways for tagging and splitting the passages. For example, we can utilize NER for entity recognitions within each passage, making the search more specific while using meta-data index approaches (allowing the search to "project" and return results containing the keywords extracted from the query). Since NER or other tagging approaches are Semantic but the search is based on Lexical matching, this approaches can be considered as both Semanticcs and Lexical.
2. Reranking approaches (Both Semantic Search and Lexical Retrieval): After retrieving top-k passages with MiniLM, apply a BERT-based cross-encoder that scores query-passage pairs for relevance. This applies to Semantic Search. We can utilize other ways Lexical approaches for reranking (for example, HITS algorithm).
3. RAG (Retrieval-Augmented Generation): Combine retrieval with LLM-based generation by incorporating retrieved chunks into a final answer. This applies primarily for semantic search. Retrieval provides context for the LLM to generate more accurate, context-aware responses.
4. Fine-tune LLMs (e.g., BERT, MiniLM) on a domain-specific corpus to improve the quality of passage embeddings.
This improves semantic search accuracy by aligning embeddings more closely with the specific task.


##### Limitations and Drawbacks:
1. Computational Overhead: Using large-scale LLMs, especially in real-time retrieval systems, increases latency and computational costs.
2. Hallucination Risk (for Generative Retrieval): Known drawback of LLMs' applications, where LLMs may hallucinate non-factual information, which can result in unreliable retrieval outputs (if not properly constrained).



## Q2 - Creating a Reliable Ground Truth for Evaluation
To create a reliable ground truth for evaluation, we need to manually annotate the dataset with relevant passages for each query.

The main limitation is the need in defining Annotation Schema, and label each chunk with some relevance score (for example, binary score for relevance or graded relevance by defining order over the relevant passages).
If tagging each chunk manually, we can use Active Learning methods for making human-annotations more efficient (Especially if we consider domain experts annotators).
An interesting approach is to define more-comprehensive ways for tagging and splitting the passages, as suggested in Q1:
- NER for entity recognitions within each passage, making the search more specific while using meta-data index approaches (allowing the search to "project" and return results containing the keywords extracted from the query).


